In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean-dirty-water-dataset/water images/test/Clean-samples/30.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Clean-samples/38.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Clean-samples/33.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Clean-samples/37.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Clean-samples/29.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Clean-samples/31.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Clean-samples/32.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Clean-samples/39.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Clean-samples/36.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Dirty-samples/16.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Dirty-samples/17.jpg
/kaggle/input/clean-dirty-water-dataset/water images/test/Dirty-samples/15.jpg
/kaggle/input/clean-dirty-water-dataset/water images

In [2]:
import os
import shutil
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
# Dataset location
elvina_root = '/kaggle/input/clean-dirty-water-dataset/water images'

# New organized dataset path
base_path = '/kaggle/working/elvina_data'
os.makedirs(base_path, exist_ok=True)

# Copy Elvina clean & dirty data
shutil.copytree(os.path.join(elvina_root, 'train', 'Clean-samples'), os.path.join(base_path, 'train/clean'))
shutil.copytree(os.path.join(elvina_root, 'train', 'Dirty-samples'), os.path.join(base_path, 'train/dirty'))
shutil.copytree(os.path.join(elvina_root, 'test', 'Clean-samples'), os.path.join(base_path, 'test/clean'))
shutil.copytree(os.path.join(elvina_root, 'test', 'Dirty-samples'), os.path.join(base_path, 'test/dirty'))


'/kaggle/working/elvina_data/test/dirty'

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
}

image_datasets = {
    x: datasets.ImageFolder(os.path.join(base_path, x), transform=data_transforms[x])
    for x in ['train', 'test']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=16, shuffle=True)
    for x in ['train', 'test']
}

class_names = image_datasets['train'].classes
print("Classes:", class_names)


Classes: ['clean', 'dirty']


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Binary classification (clean, dirty)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 168MB/s]


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5  # You can increase this for better accuracy
for epoch in range(num_epochs):
    model.train()
    running_loss, running_corrects = 0.0, 0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)

    epoch_acc = running_corrects.double() / len(image_datasets['train'])
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss:.4f} - Acc: {epoch_acc:.4f}")


Epoch 1/5 - Loss: 1.2333 - Acc: 0.7234
Epoch 2/5 - Loss: 1.5411 - Acc: 0.8511
Epoch 3/5 - Loss: 1.7039 - Acc: 0.8723
Epoch 4/5 - Loss: 0.4897 - Acc: 0.9149
Epoch 5/5 - Loss: 1.5073 - Acc: 0.8511


In [7]:
from torchvision.models import mobilenet_v2

# Load MobileNetV2 as second model
model2 = mobilenet_v2(pretrained=True)
model2.classifier[1] = nn.Linear(model2.last_channel, num_classes)  # Replace final layer

model2 = model2.to(device)

# Train model2 (copy same training loop as before)
criterion = nn.CrossEntropyLoss()
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    model2.train()
    running_loss, running_corrects = 0.0, 0
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer2.zero_grad()
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer2.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)

    epoch_acc = running_corrects.double() / len(image_datasets['train'])
    print(f"[MobileNet] Epoch {epoch+1}/{num_epochs} - Loss: {running_loss:.4f} - Acc: {epoch_acc:.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 157MB/s]


NameError: name 'num_classes' is not defined

In [ ]:
from torchvision.models import mobilenet_v2
import torch.nn as nn

num_classes = len(class_names)  # Or manually set to 2 if using Elvina clean/dirty dataset

# Load MobileNetV2 as second model
model2 = mobilenet_v2(pretrained=True)
model2.classifier[1] = nn.Linear(model2.last_channel, num_classes)

model2 = model2.to(device)

# Define optimizer and loss
criterion2 = nn.CrossEntropyLoss()
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

# Train MobileNetV2
num_epochs = 5
for epoch in range(num_epochs):
    model2.train()
    running_loss, running_corrects = 0.0, 0

    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer2.zero_grad()
        outputs = model2(inputs)
        loss = criterion2(outputs, labels)
        loss.backward()
        optimizer2.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)

    epoch_acc = running_corrects.double() / len(image_datasets['train'])
    print(f"[MobileNetV2] Epoch {epoch+1}/{num_epochs} - Loss: {running_loss:.4f} - Acc: {epoch_acc:.4f}")


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tqdm import tqdm

# Function to extract features
def extract_features(model, dataloader):
    model.eval()
    features, labels_all = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())

    features = np.vstack(features)
    labels_all = np.array(labels_all)
    return features, labels_all

# Remove softmax layer by using only the penultimate outputs
model1.eval()
model2.eval()

# Replace final classifier temporarily to get penultimate outputs
resnet_feat_extractor = nn.Sequential(*list(model.children())[:-1])
mobilenet_feat_extractor = nn.Sequential(*list(model2.children())[:-1])

def get_flat_features(model, dataloader):
    model.eval()
    flat_features, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            out = model(inputs)
            out = torch.flatten(out, 1)
            flat_features.append(out.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.vstack(flat_features), np.array(all_labels)

# Extract features from both models
features1_train, labels_train = get_flat_features(resnet_feat_extractor, dataloaders['train'])
features2_train, _ = get_flat_features(mobilenet_feat_extractor, dataloaders['train'])

features1_test, labels_test = get_flat_features(resnet_feat_extractor, dataloaders['val'])
features2_test, _ = get_flat_features(mobilenet_feat_extractor, dataloaders['val'])

# Combine features
X_train_stack = np.hstack([features1_train, features2_train])
X_test_stack = np.hstack([features1_test, features2_test])


In [ ]:
# Remove softmax layer by using only the penultimate outputs
model.eval()           # this is ResNet
model2.eval()          # this is MobileNet

# Get all layers except final FC layer
resnet_feat_extractor = nn.Sequential(*list(model.children())[:-1])
mobilenet_feat_extractor = nn.Sequential(*list(model2.children())[:-1])

def get_flat_features(model, dataloader):
    model.eval()
    flat_features, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            out = model(inputs)
            out = torch.flatten(out, 1)
            flat_features.append(out.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return np.vstack(flat_features), np.array(all_labels)

# Extract features from both models
features1_train, labels_train = get_flat_features(resnet_feat_extractor, dataloaders['train'])
features2_train, _ = get_flat_features(mobilenet_feat_extractor, dataloaders['train'])

features1_test, labels_test = get_flat_features(resnet_feat_extractor, dataloaders['val'])
features2_test, _ = get_flat_features(mobilenet_feat_extractor, dataloaders['val'])

# Combine features
X_train_stack = np.hstack([features1_train, features2_train])
X_test_stack = np.hstack([features1_test, features2_test])


In [ ]:
# Extract flat features from test set (using 'test' instead of 'val')
features1_test, labels_test = get_flat_features(resnet_feat_extractor, dataloaders['test'])
features2_test, _ = get_flat_features(mobilenet_feat_extractor, dataloaders['test'])

# Stack features from both models
X_train = np.hstack([features1_train, features2_train])
X_test = np.hstack([features1_test, features2_test])

# Train meta-classifier (Logistic Regression)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

meta_model = LogisticRegression(max_iter=1000)
meta_model.fit(X_train, labels_train)
y_pred = meta_model.predict(X_test)

# Evaluation
print("Classification Report:\n", classification_report(labels_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(labels_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt

# Helper function to display image and prediction
def visualize_predictions(meta_model, feat_extractor1, feat_extractor2, dataloader, class_names, num_images=6):
    model1.eval()
    model2.eval()

    images_shown = 0
    plt.figure(figsize=(12, 8))

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.numpy()

            feats1 = feat_extractor1(inputs).cpu().numpy()
            feats2 = feat_extractor2(inputs).cpu().numpy()
            combined_feats = np.hstack([feats1, feats2])

            preds = meta_model.predict(combined_feats)

            for i in range(inputs.size(0)):
                if images_shown >= num_images:
                    break
                img = inputs[i].cpu().permute(1, 2, 0).numpy()
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # unnormalize
                img = np.clip(img, 0, 1)

                plt.subplot(2, 3, images_shown + 1)
                plt.imshow(img)
                plt.title(f"Pred: {class_names[preds[i]]}\nTrue: {class_names[labels[i]]}")
                plt.axis('off')

                images_shown += 1
            if images_shown >= num_images:
                break
    plt.tight_layout()
    plt.show()

# Call the function to visualize predictions
visualize_predictions(meta_model, resnet_feat_extractor, mobilenet_feat_extractor, dataloaders['test'], class_names)


In [ ]:
import matplotlib.pyplot as plt

# Helper function to display predictions
def visualize_predictions(meta_model, feat_extractor1, feat_extractor2, dataloader, class_names, num_images=6):
    images_shown = 0
    plt.figure(figsize=(12, 8))

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.numpy()

            feats1 = feat_extractor1(inputs).cpu().numpy()
            feats2 = feat_extractor2(inputs).cpu().numpy()
            combined_feats = np.hstack([feats1, feats2])

            preds = meta_model.predict(combined_feats)

            for i in range(inputs.size(0)):
                if images_shown >= num_images:
                    break
                img = inputs[i].cpu().permute(1, 2, 0).numpy()
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # unnormalize
                img = np.clip(img, 0, 1)

                plt.subplot(2, 3, images_shown + 1)
                plt.imshow(img)
                plt.title(f"Pred: {class_names[preds[i]]}\nTrue: {class_names[labels[i]]}")
                plt.axis('off')

                images_shown += 1
            if images_shown >= num_images:
                break
    plt.tight_layout()
    plt.show()

# ✅ Call the function
visualize_predictions(meta_model, resnet_feat_extractor, mobilenet_feat_extractor, dataloaders['test'], class_names)


In [ ]:
import matplotlib.pyplot as plt

# Updated visualization function
def visualize_predictions(meta_model, feat_extractor1, feat_extractor2, dataloader, class_names, num_images=6):
    images_shown = 0
    plt.figure(figsize=(12, 8))

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.numpy()

            # Get and flatten features
            feats1 = feat_extractor1(inputs).cpu()
            feats2 = feat_extractor2(inputs).cpu()

            feats1_flat = feats1.view(feats1.size(0), -1).numpy()
            feats2_flat = feats2.view(feats2.size(0), -1).numpy()

            combined_feats = np.hstack([feats1_flat, feats2_flat])
            preds = meta_model.predict(combined_feats)

            for i in range(inputs.size(0)):
                if images_shown >= num_images:
                    break
                img = inputs[i].cpu().permute(1, 2, 0).numpy()
                img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # unnormalize
                img = np.clip(img, 0, 1)

                plt.subplot(2, 3, images_shown + 1)
                plt.imshow(img)
                plt.title(f"Pred: {class_names[preds[i]]}\nTrue: {class_names[labels[i]]}")
                plt.axis('off')

                images_shown += 1
            if images_shown >= num_images:
                break
    plt.tight_layout()
    plt.show()

# ✅ Call it
visualize_predictions(meta_model, resnet_feat_extractor, mobilenet_feat_extractor, dataloaders['test'], class_names)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 1. Confusion Matrix
cm = confusion_matrix(true_labels, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()

# 2. Correct vs Incorrect Predictions
correct = sum(np.array(true_labels) == np.array(pred_labels))
incorrect = len(true_labels) - correct

plt.figure(figsize=(6,4))
sns.barplot(x=["Correct", "Incorrect"], y=[correct, incorrect], palette="pastel")
plt.title("Correct vs Incorrect Predictions")
plt.ylabel("Number of Samples")
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score

# Switch to evaluation mode for feature extractors
resnet_feat_extractor.eval()
mobilenet_feat_extractor.eval()

# Extract features from test set
features1_test, labels_test = get_flat_features(resnet_feat_extractor, dataloaders['test'])
features2_test, _ = get_flat_features(mobilenet_feat_extractor, dataloaders['test'])

# Stack features
X_test = np.hstack([features1_test, features2_test])

# Predict using the trained meta-model
y_pred = meta_model.predict(X_test)

# Compute accuracy
acc = accuracy_score(labels_test, y_pred)
print(f"✅ Test Accuracy of Stacked Model: {acc * 100:.2f}%")


In [ ]:
print("True Labels:", true_labels)
print("Predicted Labels:", pred_labels)
print("Number of True Labels:", len(true_labels))
print("Number of Predicted Labels:", len(pred_labels))


In [ ]:
# Ensure model is in evaluation mode
resnet_feat_extractor.eval()
mobilenet_feat_extractor.eval()

true_labels = []
pred_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(dataloaders['test']):
        inputs = inputs.to(device)
        feats1 = resnet_feat_extractor(inputs).cpu().numpy()
        feats2 = mobilenet_feat_extractor(inputs).cpu().numpy()

        # Match shapes if needed (flatten or average pool if needed)
        if len(feats1.shape) > 2:
            feats1 = feats1.reshape(feats1.shape[0], -1)
        if len(feats2.shape) > 2:
            feats2 = feats2.reshape(feats2.shape[0], -1)

        combined_feats = np.hstack([feats1, feats2])
        preds = meta_model.predict(combined_feats)

        pred_labels.extend(preds)
        true_labels.extend(labels.numpy())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Confusion Matrix
cm = confusion_matrix(true_labels, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()

# Correct vs Incorrect Bar Plot
correct = sum(np.array(true_labels) == np.array(pred_labels))
incorrect = len(true_labels) - correct

plt.figure(figsize=(6,4))
sns.barplot(x=["Correct", "Incorrect"], y=[correct, incorrect], palette="pastel")
plt.title("Correct vs Incorrect Predictions")
plt.ylabel("Number of Samples")
plt.show()
